<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone git@github.com:Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git

In [ ]:
!pip install -U -q ivy mlflow datasets>=2.14.5 nlp 2>/dev/null
import ivy